In [1]:
%load_ext autoreload
%autoreload 

In [2]:
import sys
sys.path.append("..")

In [3]:
from optimus import Optimus

In [4]:
# Create optimus
op = Optimus()


                             ____        __  _                     
                            / __ \____  / /_(_)___ ___  __  _______
                           / / / / __ \/ __/ / __ `__ \/ / / / ___/
                          / /_/ / /_/ / /_/ / / / / / / /_/ (__  ) 
                          \____/ .___/\__/_/_/ /_/ /_/\__,_/____/  
                              /_/                                  
                              
Transform and Roll out...
Just checking that all necessary environments vars are present...
-----
HADOOP_HOME=C:\opt\spark\spark-2.3.1-bin-hadoop2.7
PYSPARK_PYTHON=python
SPARK_HOME=C:\opt\spark\spark-2.3.1-bin-hadoop2.7
JAVA_HOME=C:\Program Files\Java\jdk1.8.0_181
Pyarrow Installed
-----
Starting or getting SparkSession and SparkContext...
Optimus successfully imported. Have fun :).


## Create dataframe
### Spark

This is ugly:

```
val someData = Seq(
  Row(8, "bat"),
  Row(64, "mouse"),
  Row(-27, "horse")
)

val someSchema = List(
  StructField("number", IntegerType, true),
  StructField("word", StringType, true)
)

val someDF = spark.createDataFrame(
  spark.sparkContext.parallelize(someData),
  StructType(someSchema)
)```

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, ArrayType

df = op.create.df(
            [
                ("words", "str", True),
                ("num", "int", True),
                ("animals", "str", True),
                ("thing", StringType(), True),
                ("two strings", StringType(), True),
                ("filter", StringType(), True),
                ("num 2", "string", True),
                ("col_array",  ArrayType(StringType()), True),
                ("col_int",  ArrayType(IntegerType()), True)

            ]
,
[
                ("  I like     fish  ", 1, "dog", "housé", "cat-car", "a","1",["baby", "sorry"],[1,2,3]),
                ("    zombies", 2, "cat", "tv", "dog-tv", "b","2",["baby 1", "sorry 1"],[3,4]),
                ("simpsons   cat lady", 2, "frog", "table","eagle-tv-plus","1","3", ["baby 2", "sorry 2"], [5,6,7]),
                (None, 3, "eagle", "glass", "lion-pc", "c","4", ["baby 3", "sorry 3"] ,[7,8])
            ])

df.table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]"
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]"
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]"
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]"


## Create Columns
### Spark
* You can not create multiple columns at the same time
* You need to use the lit function. lit???

### Pandas
* Assing function seems to do the job https://stackoverflow.com/questions/12555323/adding-new-column-to-existing-dataframe-in-python-pandas


In [6]:
df = df.cols.append("new_col_1", 1)
df.table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


In [7]:
from pyspark.sql.functions import *

df.cols.append([
    ("new_col_2", 2.22),
    ("new_col_3", lit(3))
    ]).table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int),new_col_2 (double),new_col_3 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1,2.22,3
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1,2.22,3
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1,2.22,3
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1,2.22,3


In [8]:
from pyspark.sql.functions import *

df.cols.append([
    ("new_col_4", "test"),
    ("new_col_5", df['num']*2),
    ("new_col_6", [1,2,3])
    ]).table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int),new_col_4 (string),new_col_5 (int),new_col_6 (array<int>)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1,test,2,"[1,⸱2,⸱3]"
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1,test,4,"[1,⸱2,⸱3]"
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1,test,4,"[1,⸱2,⸱3]"
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1,test,6,"[1,⸱2,⸱3]"


## Select columns
### Spark
* You can not select columns by string and index at the same time

### Pandas
* You can not select columns by string and index at the same time

In [9]:
df.table()
columns = ["words", 1, "animals", 3]
df.cols.select(columns).table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


words (string),num (int),animals (string),thing (string)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé
⸱⸱⸱⸱zombies,2,cat,tv
simpsons⸱⸱⸱cat⸱lady,2,frog,table
None,3,eagle,glass


In [12]:
df.cols.select("n.*", regex = True).table()

num (int),num 2 (string),new_col_1 (int)
1,1,1
2,2,1
2,3,1
3,4,1


In [14]:
df.cols.select("*", data_type = "str").table()
df.table()

animals (string),num 2 (string),two strings (string),thing (string),filter (string),words (string)
dog,1,cat-car,housé,a,⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱
cat,2,dog-tv,tv,b,⸱⸱⸱⸱zombies
frog,3,eagle-tv-plus,table,1,simpsons⸱⸱⸱cat⸱lady
eagle,4,lion-pc,glass,c,None


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


## Rename Column
### Spark
You can not rename multiple columns using Spark Vanilla API


### Pandas
* Almost the same behavior https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rename.html

In [15]:
df.cols.rename('num','number').table()

words (string),number (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


In [16]:
df.cols.rename([('num','number'),("animals","gods")], str.upper).table()

WORDS (string),NUM (int),ANIMALS (string),THING (string),TWO STRINGS (string),FILTER (string),NUM 2 (string),COL_ARRAY (array<string>),COL_INT (array<int>),NEW_COL_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


In [17]:
df.cols.rename(str.lower).table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


In [18]:
df.cols.rename(str.upper).table()

WORDS (string),NUM (int),ANIMALS (string),THING (string),TWO STRINGS (string),FILTER (string),NUM 2 (string),COL_ARRAY (array<string>),COL_INT (array<int>),NEW_COL_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


## Cast a columns

### Spark
* Can not cast multiple columns

### Pandas
This is a opinionated way to handle column casting. 
One of the first thing that every data cleaning process need to acomplish is define a data dictionary.
Because of that we prefer to create a tuple like this:

df.cols().cast(
[("words","str"),
("num","int"),
("animals","float"),
("thing","str")]
)


https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.astype.html

In [19]:
df.cols.cast([("num", "string"),("num 2", "integer")]).dtypes

[('words', 'string'),
 ('num', 'string'),
 ('animals', 'string'),
 ('thing', 'string'),
 ('two strings', 'string'),
 ('filter', 'string'),
 ('num 2', 'int'),
 ('col_array', 'array<string>'),
 ('col_int', 'array<int>'),
 ('new_col_1', 'int')]

In [20]:
df.cols.cast("num", "string").dtypes

[('words', 'string'),
 ('num', 'string'),
 ('animals', 'string'),
 ('thing', 'string'),
 ('two strings', 'string'),
 ('filter', 'string'),
 ('num 2', 'string'),
 ('col_array', 'array<string>'),
 ('col_int', 'array<int>'),
 ('new_col_1', 'int')]

In [21]:
df.cols.cast("*", "string").dtypes

[('words', 'string'),
 ('num', 'string'),
 ('animals', 'string'),
 ('thing', 'string'),
 ('two strings', 'string'),
 ('filter', 'string'),
 ('num 2', 'string'),
 ('col_array', 'string'),
 ('col_int', 'string'),
 ('new_col_1', 'string')]

In [22]:
list(zip(['words', 'num', 'animals', 'thing', 'two strings', 'filter', 'num 2', 'col_array', 'col_int'], ('string', 'string', 'string', 'string', 'string', 'string', 'string', 'string', 'string')))

[('words', 'string'),
 ('num', 'string'),
 ('animals', 'string'),
 ('thing', 'string'),
 ('two strings', 'string'),
 ('filter', 'string'),
 ('num 2', 'string'),
 ('col_array', 'string'),
 ('col_int', 'string')]

In [23]:
from pyspark.ml.linalg import Vectors

df.cols.cast("col_int", Vectors)

DataFrame[words: string, num: int, animals: string, thing: string, two strings: string, filter: string, num 2: string, col_array: array<string>, col_int: vector, new_col_1: int]

## Keep columns
### Spark
* You can not remove multiple columns

### Pandas
* Handle in pandas with drop


In [24]:
from pyspark.sql.functions import *
df.withColumn("num", col("num").cast(StringType()))


DataFrame[words: string, num: string, animals: string, thing: string, two strings: string, filter: string, num 2: string, col_array: array<string>, col_int: array<int>, new_col_1: int]

In [25]:
df.table()
df.cols.keep("num").table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


num (int)
1
2
2
3


## Move columns
### Spark
Do not exist in spark

### Pandas
Do not exist in pandas

In [27]:
df.cols.move("words", "after", "thing").table()

num (int),animals (string),thing (string),words (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
1,dog,housé,⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
2,cat,tv,⸱⸱⸱⸱zombies,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
2,frog,table,simpsons⸱⸱⸱cat⸱lady,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
3,eagle,glass,None,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


## Sorting Columns
### Spark
You can not sort columns using Spark Vanilla API 

### Pandas
Similar to pandas
http://pandas.pydata.org/pandas-docs/version/0.19/generated/pandas.DataFrame.sort_values.html#pandas.DataFrame.sort_values

In [28]:
df.cols.sort().table()

animals (string),col_array (array<string>),col_int (array<int>),filter (string),new_col_1 (int),num (int),num 2 (string),thing (string),two strings (string),words (string)
dog,"['baby',⸱'sorry']","[1,⸱2,⸱3]",a,1,1,1,housé,cat-car,⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱
cat,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",b,1,2,2,tv,dog-tv,⸱⸱⸱⸱zombies
frog,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1,1,2,3,table,eagle-tv-plus,simpsons⸱⸱⸱cat⸱lady
eagle,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",c,1,3,4,glass,lion-pc,None


In [29]:
df.cols.sort(order = "desc").table()

words (string),two strings (string),thing (string),num 2 (string),num (int),new_col_1 (int),filter (string),col_int (array<int>),col_array (array<string>),animals (string)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,cat-car,housé,1,1,1,a,"[1,⸱2,⸱3]","['baby',⸱'sorry']",dog
⸱⸱⸱⸱zombies,dog-tv,tv,2,2,1,b,"[3,⸱4]","['baby⸱1',⸱'sorry⸱1']",cat
simpsons⸱⸱⸱cat⸱lady,eagle-tv-plus,table,3,2,1,1,"[5,⸱6,⸱7]","['baby⸱2',⸱'sorry⸱2']",frog
None,lion-pc,glass,4,3,1,c,"[7,⸱8]","['baby⸱3',⸱'sorry⸱3']",eagle


## Drop columns
### Spark 
* You can not delete multiple colums

### Pandas
* Almost the same as pandas
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html

In [30]:
df2 = df.cols.drop("num")
df2 = df.cols.drop(["num","words"])
df2.table()

animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


In [31]:
df.table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


## Chaining

cols y rows functions are used to organize and encapsulate optimus' functionality apart of Apache Spark Dataframe API. This have a disadvantage at chaining time because we need to user invoke cols or rows in every step.

At the same time it can be helpfull when you look at the code because every line is self explained.

In [32]:
df\
    .cols.rename([('num','number')])\
    .cols.drop(["number","words"])\
    .withColumn("new_col_2", lit("spongebob"))\
    .cols.append("new_col_1", 1)\
    .cols.sort(order= "desc")\
    .table()

two strings (string),thing (string),num 2 (string),new_col_2 (string),new_col_1 (int),filter (string),col_int (array<int>),col_array (array<string>),animals (string)
cat-car,housé,1,spongebob,1,a,"[1,⸱2,⸱3]","['baby',⸱'sorry']",dog
dog-tv,tv,2,spongebob,1,b,"[3,⸱4]","['baby⸱1',⸱'sorry⸱1']",cat
eagle-tv-plus,table,3,spongebob,1,1,"[5,⸱6,⸱7]","['baby⸱2',⸱'sorry⸱2']",frog
lion-pc,glass,4,spongebob,1,c,"[7,⸱8]","['baby⸱3',⸱'sorry⸱3']",eagle


## Unnest  Columns
### Spark

### Pandas

In [33]:
df.table()
df.cols.unnest("two strings","-")\
    .table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int),two strings_0 (string),two strings_1 (string)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1,cat,car
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1,dog,tv
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1,eagle,tv
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1,lion,pc


In [34]:
df.cols.unnest("two strings","-", index = 1).table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int),two strings_1 (string)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1,car
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1,tv
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1,tv
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1,pc


## Impute
### Spark

In [35]:
df_cast =df.cols.cast([("num","double"),("num 2", "double")])
df_cast.dtypes

[('words', 'string'),
 ('num', 'double'),
 ('animals', 'string'),
 ('thing', 'string'),
 ('two strings', 'string'),
 ('filter', 'string'),
 ('num 2', 'double'),
 ('col_array', 'array<string>'),
 ('col_int', 'array<int>'),
 ('new_col_1', 'int')]

In [36]:
df_cast.cols.impute(["num","num 2"], ["out_a","out_B"], "mean")
df_cast.table()

words (string),num (double),animals (string),thing (string),two strings (string),filter (string),num 2 (double),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1.0,dog,housé,cat-car,a,1.0,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2.0,cat,tv,dog-tv,b,2.0,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2.0,frog,table,eagle-tv-plus,1,3.0,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3.0,eagle,glass,lion-pc,c,4.0,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


## Get columns by type
### Spark

### Pandas

In [37]:
df.cols.filter_by_dtypes("int").table()

new_col_1 (int),num (int)
1,1
1,2
1,2
1,3


## Apply custom function
### Spark
You need to declare a UDF Spark function

### Pandas
Almost the same behavior

In [38]:
df.table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


In [39]:
def sum_(val, attr):
    return attr

df.cols.apply_by_dtypes("filter", sum_, "string", "10", data_type="integer").table()

Using 'pandas_udf' to process column 'filter' with function sum_


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,10,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


In [40]:
def func(val, attr):
    return val + attr

df.cols.apply(["num", "num"], func, "int", 32 ,"udf").table()

Using 'udf' to process column 'num' with function func
Using 'udf' to process column 'num' with function func


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,65,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,66,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,66,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,67,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


In [41]:
from optimus.functions import filter_row_by_data_type as fbdt

df.where(fbdt("filter", "integer")).table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1


In [42]:
from optimus.functions import abstract_udf as audf 

def func(val, attr):
    return val>1

df.where(audf("num", func, "boolean")).table()

Using 'pandas_udf' to process column 'num' with function func


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


In [43]:
def func(val, attr):
    return val + attr

df.cols.apply(["num", "num"], func, "int", 10).table()

Using 'pandas_udf' to process column 'num' with function func
Using 'pandas_udf' to process column 'num' with function func


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,21,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,22,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,22,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,23,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


In [44]:
# https://stackoverflow.com/questions/31400143/column-filtering-in-pyspark
from optimus.functions import abstract_udf as audf 

def func(val, attr):
    return val+attr[0]

df.withColumn("num", audf ("num", func, "int", [10,20])).table()

Using 'pandas_udf' to process column 'num' with function func


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,11,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,12,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,12,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,13,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


In [45]:
from pyspark.sql import functions as F
def func(col_name, attr):
    # return F.col(col_name) + 1
    return F.when(F.col(col_name)>0 ,2)

df.cols.apply_exp(["num","num 2"], func).table()

Using 'column_exp' to process column 'num' with function func
Using 'column_exp' to process column 'num 2' with function func


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (int),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,2,dog,housé,cat-car,a,2,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,2,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,2,eagle,glass,lion-pc,c,2,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


In [46]:
from pyspark.sql import functions as F
def func(col_name, attr):
    return F.upper(F.col(col_name))

df.cols.apply_exp(["two strings","animals"], func).table()

Using 'column_exp' to process column 'two strings' with function func
Using 'column_exp' to process column 'animals' with function func


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,DOG,housé,CAT-CAR,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,CAT,tv,DOG-TV,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,FROG,table,EAGLE-TV-PLUS,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,EAGLE,glass,LION-PC,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


In [47]:
def func(val, attr):
    return 10

col = "num"

df.cols.apply(col, func, "int", when= df["num"]>1).table()

df.cols.apply(col, func, "int", when= fbdt(col, "int")).table()

Using 'pandas_udf' to process column 'num' with function func


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,10,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,10,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,10,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


Using 'pandas_udf' to process column 'num' with function func


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,10,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,10,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,10,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,10,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


## Count Nulls
### Spark

### Pandas

In [48]:
import numpy as np

df_null = op.spark.createDataFrame(
    [(1, 1, None), (1, 2, float(5)), (1, 3, np.nan), (1, 4, None), (1, 5, float(10)), (1, 6, float('nan')), (1, 6, float('nan'))],
    ('session', "timestamp1", "id2"))

In [49]:
df_null.cols.count_na("id2")

5

In [50]:
df_null.cols.count_na("*")

{'session': 0, 'timestamp1': 0, 'id2': 5}

## Count uniques
### Spark

### Pandas


In [53]:
df.cols.count_uniques("*")

AnalysisException: "cannot resolve 'CAST(`col_array` AS FLOAT)' due to data type mismatch: cannot cast array<string> to float;;\n'Project [words#1805, num#1816, animals#1827, thing#1838, two strings#1849, filter#1860, num 2#1871, cast(col_array#7 as float) AS col_array#1882, col_int#8, new_col_1#42]\n+- AnalysisBarrier\n      +- Project [words#1805, num#1816, animals#1827, thing#1838, two strings#1849, filter#1860, cast(num 2#6 as float) AS num 2#1871, col_array#7, col_int#8, new_col_1#42]\n         +- Project [words#1805, num#1816, animals#1827, thing#1838, two strings#1849, cast(filter#5 as float) AS filter#1860, num 2#6, col_array#7, col_int#8, new_col_1#42]\n            +- Project [words#1805, num#1816, animals#1827, thing#1838, cast(two strings#4 as float) AS two strings#1849, filter#5, num 2#6, col_array#7, col_int#8, new_col_1#42]\n               +- Project [words#1805, num#1816, animals#1827, cast(thing#3 as float) AS thing#1838, two strings#4, filter#5, num 2#6, col_array#7, c

## Unique
### Spark
An abstraction of distinct to be use in multiple columns at the same time

### Pandas
Similar behavior than pandas

In [54]:
df.table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


In [55]:
df_distinct = op.create.df(
            [
                ("words", "str", True),
                ("num", "int", True)
            ],
[
                ("  I like     fish  ", 1),
                ("    zombies", 2),
                ("simpsons   cat lady", 2),
                (None, 3),
                  (None, 0)
            ])

In [56]:
df_distinct.cols.unique("num").table()

num (int)
1
3
2
0


## Count Zeros

In [57]:
df_zeros = df_distinct
df_zeros.table()
df_zeros.cols.count_zeros("*")

words (string),num (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1
⸱⸱⸱⸱zombies,2
simpsons⸱⸱⸱cat⸱lady,2
None,3
None,0


{'words': 0, 'num': 1}

## Column Data Types

In [58]:
df.cols.dtypes('*')

{'words': 'string',
 'num': 'int',
 'animals': 'string',
 'thing': 'string',
 'two strings': 'string',
 'filter': 'string',
 'num 2': 'string',
 'col_array': 'array<string>',
 'col_int': 'array<int>',
 'new_col_1': 'int'}

## Replace

In [59]:
df.cols.replace(["two strings","animals"], ["dog-tv", "cat", "eagle", "fish"], "animals").table()
df.cols.replace('animals',["dog","cat"],"animals").table()

df.cols.replace('animals',[("dog","animals"),("cat","animals")]).table()

df.cols.replace('animals',"dog",'animal').table()

df.cols.replace('num',["3",2], 10).table()

df.cols.replace('num',[("3",6),(2,6)]).table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,animals,tv,animals,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,animals,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,animals,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,animals,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,animals,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,animals,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,animal,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


In [60]:
df.cols.replace('*','.*[Cc]at.*', 'animal', regex=True).table()
#df.cols().replace('*','cat', 'animal', regex=True).show()


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,animal,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,animal,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
animal,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


## Nest

In [61]:
df.cols.nest(["num", "num"], "nested","sdf" ,"vector").table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int),nested (vector)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1,"[1.0,1.0]"
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1,"[2.0,2.0]"
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1,"[2.0,2.0]"
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1,"[3.0,3.0]"


In [62]:
df.cols.nest(["animals", "two strings"], ["nested"], shape = "string").table()

Using 'column_exp' to process column 'nested' with function func_col_exp


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int),nested (string)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1,None
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1,None
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1,None
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1,None


In [63]:
df.cols.nest(["animals", "two strings"], ["nested"],shape="array").table()

Using 'column_exp' to process column 'nested' with function func_col_exp


words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int),nested (array<string>)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1,"['dog',⸱'cat-car']"
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1,"['cat',⸱'dog-tv']"
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1,"['frog',⸱'eagle-tv-plus']"
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1,"['eagle',⸱'lion-pc']"


## Unnest
Works with arrays, Vectors and strings(aka split)

In [65]:
df\
    .cols.unnest(["col_array"])\
    .cols.unnest(["col_int"])\
    .cols.unnest(["two strings"], n= 3, mark = "-")\
    .cols.select("two.*", regex = True)\
    .table()

thing (string),two strings (string),two strings_0 (string),two strings_1 (string),two strings_2 (string)
housé,cat-car,cat,car,None
tv,dog-tv,dog,tv,None
table,eagle-tv-plus,eagle,tv,plus
glass,lion-pc,lion,pc,None


In [66]:
from pyspark.ml.linalg import Vectors

df1 = op.sc.parallelize([
    ("assert", Vectors.dense([1, 2, 3])),
    ("require", Vectors.sparse(3, {1: 2}))
]).toDF(["word", "vector"])    

In [67]:
df1\
    .cols.unnest(["vector"])\
    .table()

word (string),vector (vector),_3 (double),_4 (double),_5 (double)
assert,"[1.0,2.0,3.0]",1.0,2.0,3.0
require,"(3,[1],[2.0])",0.0,2.0,0.0


## Hist

In [69]:
df.cols.hist("num 2")

TypeError: hist() missing 2 required positional arguments: 'min_value' and 'max_value'

In [70]:
df.table()

words (string),num (int),animals (string),thing (string),two strings (string),filter (string),num 2 (string),col_array (array<string>),col_int (array<int>),new_col_1 (int)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog,housé,cat-car,a,1,"['baby',⸱'sorry']","[1,⸱2,⸱3]",1
⸱⸱⸱⸱zombies,2,cat,tv,dog-tv,b,2,"['baby⸱1',⸱'sorry⸱1']","[3,⸱4]",1
simpsons⸱⸱⸱cat⸱lady,2,frog,table,eagle-tv-plus,1,3,"['baby⸱2',⸱'sorry⸱2']","[5,⸱6,⸱7]",1
None,3,eagle,glass,lion-pc,c,4,"['baby⸱3',⸱'sorry⸱3']","[7,⸱8]",1


## DESCRIPTIVE ANALYTICS

In [71]:
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, ArrayType
df =op.load.url("https://raw.githubusercontent.com/ironmussa/Optimus/master/examples/foo.csv")

Downloaded 967 bytes
Creating DataFrame for foo.csv. Please wait...
Loading file using SparkSession
Successfully created DataFrame for 'foo.csv'


### Quantile Statistics

In [72]:
print(df.cols.min("billingId"))
print(df.cols.percentile(['billingId', 'price'], [0.05, 0.25, 0.5, 0.75, 0.95]))
print(df.cols.max("billingId"))
print(df.cols.median(["billingId","price"]))
print(df.cols.range(["billingId","price"]))
print(df.cols.std(["billingId","price"]))

111.0
{'billingId': {0.05: 111.0, 0.25: 323.0, 0.5: 553.0, 0.75: 812.0, 0.95: 992.0}, 'price': {0.05: 1.0, 0.25: 3.0, 0.5: 8.0, 0.75: 8.0, 0.95: 10.0}}
992.0
{'billingId': 553.0, 'price': 8.0}
{'billingId': {'min': 111.0, 'max': 992.0}, 'price': {'min': 1.0, 'max': 10.0}}
{'billingId': {'stddev': 280.1973510859008}, 'price': {'stddev': 2.9528457876452054}}


In [73]:
print(df.cols.min("*"))

{'id': {'min': 1.0}, 'firstName': {'min': None}, 'lastName': {'min': None}, 'billingId': {'min': 111.0}, 'product': {'min': 110790.0}, 'price': {'min': 1.0}, 'birth': {'min': None}, 'dummyCol': {'min': None}}


In [74]:
### Descriptive Statistics

In [75]:
print(df.cols.kurt("billingId"))
print(df.cols.mean("billingId"))
print(df.cols.skewness("billingId"))
print(df.cols.sum("billingId"))
print(df.cols.variance("billingId"))
print(df.cols.mad("billingId"))

-1.0411548120618528
556.0
-0.2137018086949909
10564.0
78510.55555555555
230.0


In [76]:
df.cols.mad("price")

2.0

In [77]:
df.cols.mad("price", more= True)

{'mad': 2.0, 'median': 8.0}

In [78]:
print(df.cols.percentile(['price'], [0.05, 0.25, 0.5, 0.75, 0.95]))

{0.05: 1.0, 0.25: 3.0, 0.5: 8.0, 0.75: 8.0, 0.95: 10.0}


In [79]:
print(df.cols.mode(["price","billingId"]))

[{'price': 8}, {'billingId': None}]


## String Operations

In [80]:
df.table()

id (int),firstName (string),lastName (string),billingId (int),product (string),price (int),birth (string),dummyCol (string)
1,Luis,Alvarez$$%!,123,Cake,10,1980/07/07,never
2,André,Ampère,423,piza,8,1950/07/08,gonna
3,NiELS,Böhr//((%%,551,pizza,8,1990/07/09,give
4,PAUL,dirac$,521,pizza,8,1954/07/10,you
5,Albert,Einstein,634,pizza,8,1990/07/11,up
6,Galileo,⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱GALiLEI,672,arepa,5,1930/08/12,never
7,CaRL,Ga%%%uss,323,taco,3,1970/07/13,gonna
8,David,H$$$ilbert,624,taaaccoo,3,1950/07/14,let
9,Johannes,KEPLER,735,taco,3,1920/04/22,you
10,JaMES,M$$ax%%well,875,taco,3,1923/03/12,down


In [81]:
df\
    .cols.trim("lastName")\
    .cols.lower("lastName")\
    .cols.upper(["product", "firstName"])\
    .cols.reverse("firstName")\
    .table()

Using 'column_exp' to process column 'lastName' with function _trim
Using 'column_exp' to process column 'lastName' with function _lower
Using 'column_exp' to process column 'product' with function _upper
Using 'column_exp' to process column 'firstName' with function _upper
Using 'column_exp' to process column 'firstName' with function _reverse


id (int),firstName (string),lastName (string),billingId (int),product (string),price (int),birth (string),dummyCol (string)
1,SIUL,alvarez$$%!,123,CAKE,10,1980/07/07,never
2,ÉRDNA,ampère,423,PIZA,8,1950/07/08,gonna
3,SLEIN,böhr//((%%,551,PIZZA,8,1990/07/09,give
4,LUAP,dirac$,521,PIZZA,8,1954/07/10,you
5,TREBLA,einstein,634,PIZZA,8,1990/07/11,up
6,OELILAG,galilei,672,AREPA,5,1930/08/12,never
7,LRAC,ga%%%uss,323,TACO,3,1970/07/13,gonna
8,DIVAD,h$$$ilbert,624,TAAACCOO,3,1950/07/14,let
9,SENNAHOJ,kepler,735,TACO,3,1920/04/22,you
10,SEMAJ,m$$ax%%well,875,TACO,3,1923/03/12,down


In [82]:
df.cols.iqr("price")

5.0

In [83]:
df.cols.iqr("price", more= True)

{'iqr': 5.0, 'q1': 3.0, 'q3': 8.0}

In [84]:
df.cols.z_score("price").table()

id (int),firstName (string),lastName (string),billingId (int),product (string),price (int),birth (string),dummyCol (string),z_col_price (double)
1,Luis,Alvarez$$%!,123,Cake,10,1980/07/07,never,1.3368014129178498
2,André,Ampère,423,piza,8,1950/07/08,gonna,0.6594886970394727
3,NiELS,Böhr//((%%,551,pizza,8,1990/07/09,give,0.6594886970394727
4,PAUL,dirac$,521,pizza,8,1954/07/10,you,0.6594886970394727
5,Albert,Einstein,634,pizza,8,1990/07/11,up,0.6594886970394727
6,Galileo,⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱GALiLEI,672,arepa,5,1930/08/12,never,0.3564803767780932
7,CaRL,Ga%%%uss,323,taco,3,1970/07/13,gonna,1.0337930926564705
8,David,H$$$ilbert,624,taaaccoo,3,1950/07/14,let,1.0337930926564705
9,Johannes,KEPLER,735,taco,3,1920/04/22,you,1.0337930926564705
10,JaMES,M$$ax%%well,875,taco,3,1923/03/12,down,1.0337930926564705


In [85]:
## COLUMNS FUNCTIONS

In [86]:
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, ArrayType
df = op.create.df(
            [
                ("words", "str", True),
                ("num", "int", True),
                ("animals", "str", True),
                ("thing", StringType(), True),
                ("two strings", StringType(), True),
                ("filter", StringType(), True),
                ("num 2", "string", True),
                ("date", "string", True),
                ("num 3", "string", True)
                
            ],
[
                ("  I like     fish  ", 1, "dog", "&^%$#housé", "cat-car", "a","1", "20150510", "3"),
                ("    zombies", 2, "cat", "tv", "dog-tv", "b","2", "20160510", "3"),
                ("simpsons   cat lady", 2, "frog", "table","eagle-tv-plus","1","3", "20170510", "4"),
                (None, 3, "eagle", "glass", "lion-pc", "c","4", "20180510", "5"),
    
            ])

df.show()

+-------------------+---+-------+----------+-------------+------+-----+--------+-----+
|              words|num|animals|     thing|  two strings|filter|num 2|    date|num 3|
+-------------------+---+-------+----------+-------------+------+-----+--------+-----+
|  I like     fish  |  1|    dog|&^%$#housé|      cat-car|     a|    1|20150510|    3|
|            zombies|  2|    cat|        tv|       dog-tv|     b|    2|20160510|    3|
|simpsons   cat lady|  2|   frog|     table|eagle-tv-plus|     1|    3|20170510|    4|
|               null|  3|  eagle|     glass|      lion-pc|     c|    4|20180510|    5|
+-------------------+---+-------+----------+-------------+------+-----+--------+-----+



In [87]:
df.cols.date_transform("date", "new_date", "yyyyMMdd", "dd-MM-YYYY").show()

Using 'column_exp' to process column 'new_date' with function _date_transform
+-------------------+---+-------+----------+-------------+------+-----+--------+-----+----------+
|              words|num|animals|     thing|  two strings|filter|num 2|    date|num 3|  new_date|
+-------------------+---+-------+----------+-------------+------+-----+--------+-----+----------+
|  I like     fish  |  1|    dog|&^%$#housé|      cat-car|     a|    1|20150510|    3|10-05-2015|
|            zombies|  2|    cat|        tv|       dog-tv|     b|    2|20160510|    3|10-05-2016|
|simpsons   cat lady|  2|   frog|     table|eagle-tv-plus|     1|    3|20170510|    4|10-05-2017|
|               null|  3|  eagle|     glass|      lion-pc|     c|    4|20180510|    5|10-05-2018|
+-------------------+---+-------+----------+-------------+------+-----+--------+-----+----------+



In [88]:
df.cols.years_between("date", "new date", "yyyyMMdd",).show()

Using 'column_exp' to process column 'new date' with function _years_between
+-------------------+---+-------+----------+-------------+------+-----+--------+-----+--------+
|              words|num|animals|     thing|  two strings|filter|num 2|    date|num 3|new date|
+-------------------+---+-------+----------+-------------+------+-----+--------+-----+--------+
|  I like     fish  |  1|    dog|&^%$#housé|      cat-car|     a|    1|20150510|    3|  3.2608|
|            zombies|  2|    cat|        tv|       dog-tv|     b|    2|20160510|    3|  2.2608|
|simpsons   cat lady|  2|   frog|     table|eagle-tv-plus|     1|    3|20170510|    4|  1.2608|
|               null|  3|  eagle|     glass|      lion-pc|     c|    4|20180510|    5|  0.2608|
+-------------------+---+-------+----------+-------------+------+-----+--------+-----+--------+



In [89]:
df.cols.remove_accents("thing").show()

Using 'pandas_udf' to process column 'thing' with function _remove_accents
+-------------------+---+-------+----------+-------------+------+-----+--------+-----+
|              words|num|animals|     thing|  two strings|filter|num 2|    date|num 3|
+-------------------+---+-------+----------+-------------+------+-----+--------+-----+
|  I like     fish  |  1|    dog|&^%$#house|      cat-car|     a|    1|20150510|    3|
|            zombies|  2|    cat|        tv|       dog-tv|     b|    2|20160510|    3|
|simpsons   cat lady|  2|   frog|     table|eagle-tv-plus|     1|    3|20170510|    4|
|               null|  3|  eagle|     glass|      lion-pc|     c|    4|20180510|    5|
+-------------------+---+-------+----------+-------------+------+-----+--------+-----+



In [90]:
df.cols.remove_special_chars("thing").show()

Using 'pandas_udf' to process column 'thing' with function _remove_special_chars
+-------------------+---+-------+-----+-------------+------+-----+--------+-----+
|              words|num|animals|thing|  two strings|filter|num 2|    date|num 3|
+-------------------+---+-------+-----+-------------+------+-----+--------+-----+
|  I like     fish  |  1|    dog|housé|      cat-car|     a|    1|20150510|    3|
|            zombies|  2|    cat|   tv|       dog-tv|     b|    2|20160510|    3|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|     1|    3|20170510|    4|
|               null|  3|  eagle|glass|      lion-pc|     c|    4|20180510|    5|
+-------------------+---+-------+-----+-------------+------+-----+--------+-----+

